In [1]:
import os

In [3]:
pip install langchain==0.0.249  GitPython python-dotenv


  Using cached langchain-0.0.249-py3-none-any.whl.metadata (14 kB)
  Using cached GitPython-3.1.43-py3-none-any.whl.metadata (13 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached PyYAML-6.0.1-cp38-cp38-win_amd64.whl.metadata (2.1 kB)
  Using cached SQLAlchemy-2.0.30-cp38-cp38-win_amd64.whl.metadata (9.8 kB)
  Using cached aiohttp-3.9.5-cp38-cp38-win_amd64.whl.metadata (7.7 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached dataclasses_json-0.5.14-py3-none-any.whl.metadata (22 kB)
  Using cached langsmith-0.0.92-py3-none-any.whl.metadata (9.9 kB)
  Using cached numexpr-2.8.6-cp38-cp38-win_amd64.whl.metadata (8.2 kB)
  Using cached numpy-1.24.4-cp38-cp38-win_amd64.whl.metadata (5.6 kB)
  Using cached openapi_schema_pydantic-1.2.4-py3-none-any.whl.metadata (8.5 kB)
  Using cached pydantic-1.10.15-cp38-cp38-win_amd64.whl.metadata (152 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using ca

In [4]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

### Clone Github repositories

In [5]:
%pwd

'c:\\Users\\91900\\OneDrive\\Desktop\\Gen_AI_Projects\\MLOps_Project\\GenAI\\Source-Code-Analysis-using-GenAI\\Source-Code-Analysis-using-GenAI\\research'

In [7]:
!mkdir test_repo

In [8]:
repo_path = "test_repo/"

Repo.clone_from("https://github.com/entbappy/End-to-end-ML-Project-Implementation", to_path=repo_path)

<git.repo.base.Repo 'c:\\Users\\91900\\OneDrive\\Desktop\\Gen_AI_Projects\\MLOps_Project\\GenAI\\Source-Code-Analysis-using-GenAI\\Source-Code-Analysis-using-GenAI\\research\\test_repo\\.git'>

In [9]:
repo_path = "test_repo/"

loader = GenericLoader.from_filesystem(repo_path+'/src/mlProject',
                                        glob = "**/*",
                                       suffixes=[".py"],
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

In [10]:
documents = loader.load()

In [11]:
documents

[Document(page_content='import os\nimport sys\nimport logging\n\nlogging_str = "[%(asctime)s: %(levelname)s: %(module)s: %(message)s]"\n\nlog_dir = "logs"\nlog_filepath = os.path.join(log_dir,"running_logs.log")\nos.makedirs(log_dir, exist_ok=True)\n\n\nlogging.basicConfig(\n    level= logging.INFO,\n    format= logging_str,\n\n    handlers=[\n        logging.FileHandler(log_filepath),\n        logging.StreamHandler(sys.stdout)\n    ]\n)\n\nlogger = logging.getLogger("mlProjectLogger")', metadata={'source': 'test_repo\\src\\mlProject\\__init__.py', 'language': <Language.PYTHON: 'python'>}),
 Document(page_content='import os\nimport urllib.request as request\nimport zipfile\nfrom mlProject import logger\nfrom mlProject.utils.common import get_size\nfrom mlProject.entity.config_entity import DataIngestionConfig\nfrom pathlib import Path\n\n\nclass DataIngestion:\n    def __init__(self, config: DataIngestionConfig):\n        self.config = config\n\n    \n    def download_file(self):\n    

### Chunkings

In [10]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(language = Language.PYTHON,
                                                             chunk_size = 2000,
                                                             chunk_overlap = 200)

In [11]:
texts = documents_splitter.split_documents(documents)

In [12]:
len(texts)

19

### Embedding model

In [13]:
os.environ["OPENAI_API_KEY"] = "***************************"

In [14]:
embeddings=OpenAIEmbeddings(disallowed_special=())

### Knowledge base (vector DB)

In [15]:
vectordb = Chroma.from_documents(texts, embedding=embeddings, persist_directory='./data')
vectordb.persist()

### LLM Wrapper

In [16]:
# llm = ChatOpenAI(model_name="gpt-4")
llm = ChatOpenAI()

In [17]:
memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)

In [18]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":3}), memory=memory)

### Q&A

In [19]:
question = "what is DataIngestion class?"

In [20]:
result = qa(question)
print(result['answer'])

Number of requested results 20 is greater than number of elements in index 19, updating n_results = 19


The purpose of the `DataIngestion` class is to handle the downloading and extraction of files based on the provided configuration. It is part of the Data Ingestion stage in the training pipeline and performs tasks such as downloading files and extracting zip files.
